In [ ]:
# ON GOOGLE COLAB, THIS SECTION MUST BE INITIALIZED. IT TAKES ABOUT 2MINUTES.
# AFTER RUNNING THIS SECTION, RUNTIME(SESSION) MUST BE RESTARTED, IN ORDER TO
# USE labwons/ PACKAGE. SESSION RESTART HOTKEY IS CTRL + M.
!git clone https://github.com/labwons/labwons.git
%cd labwons
!pip install -r requirements.txt
!pip install -e .

In [2]:
import os

ACTION = os.environ.get("GITHUB_EVENT_NAME", "LOCALHOST").upper()
HOSTID = os.environ.get("USERDOMAIN", "COLAB") if ACTION == "LOCALHOST" else "GITHUB"
JOBS   = [
    "FETCH.AFTER-MARKET"
]

In [3]:
from labwons.util import DATETIME, DD

# GITHUB ACTION
if ACTION == "SCHEDULE":

    # SYSTEM EXITS WHEN THE LATEST TRADING DATE AND BUILD DATE IS DIFFERENT.
    # THIS CODE IS IMPLEMENTED IN ORDER TO AVOID RUNNING ON WEEKDAY OR HOLIDAY
    # OF THE MARKET.
    if DATETIME.TRADING != DATETIME.TODAY:
        raise SystemExit

    # IF THE TASK IS TRIGGERED BEFORE THE MARKET CLOSES, WHICH HARDLY HAPPENS,
    # SYSTEM PAUSES UNTIL THE MARKET CLOSES
    while (DATETIME.TIME().hour == 15) and (15 <= DATETIME.TIME().minute < 31):
        DATETIME.pause(30)

    if DATETIME.TIME().hour >= 20:
        JOBS = ["FETCH.ECOS", "FETCH.FRED", "FETCH.FINANCIAL-STATEMENT"]
    else:
        JOBS = ["FETCH.AFTER-MARKET", "FETCH.STOCKS"]
    JOBS += ["BUILD.MARKET-BASELINE", "BUILD.MACRO-BASELINE", "DEPLOY.ALL"]

# if ACTION == "LOCALHOST" and HOSTID == "HKEFICO":
#     JOBS = [job for job in JOBS if not job.startswith("FETCH")]

In [ ]:
from labwons.fetch import FETCH_OK, AfterMarket
from labwons.path import PATH

archive = os.path.join(PATH.ARCHIVE, DATETIME.TODAY)
dataset = DD()
if "FETCH.AFTER-MARKET" in JOBS:
    afterMarket = AfterMarket()
    if afterMarket.status == FETCH_OK:
        dataset[os.path.join(archive, afterMarket.fname)] = afterMarket.data

os.makedirs(archive, exist_ok=True)
for file, data in dataset.items():
    data.to_parquet(f"{file}.parquet", engine='pyarrow')

# TODO
# 일정 기간이 지난 날짜의 아카이브 폴더는 드라이브로 백업하는 코드 작성: Clean-UP

In [ ]:
from labwons.util import Mail
from labwons.logs import read_log

report = Mail()
report.content = content = read_log("fetch", "build")
report.subject = f'[{"FAILED" if "FAILED" in content else "SUCCESS"}] BUILD LABWONS : {DATETIME.CLOCK().strftime("%Y/%m/%d %H:%M")}'
report.send()

'\n\n'